<a href="https://colab.research.google.com/github/pcbzmani/Learning-repository-for-scala-and-pyspark/blob/main/Solutions/Python/percentage_hike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 38.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=164afa097f645359b275d9e5b50340ed52278d61528a26c475fbf164e0fcf122
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [23]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("percentagerise")\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

In [76]:
columns = ["emp_no","salary","from_date"]
data = [("10001", 60117,"2021-01-01"), ("10001", 62102,"2022-01-01"), ("10002",66074,"2021-01-01"),("10002",66596,"2022-01-01"), ("10003", 22222,"2022-01-01"), ("10004", 3000,"2022-01-01")]

In [77]:
rdd = spark.sparkContext.parallelize(data)

In [78]:
df = spark.createDataFrame(rdd).toDF(*columns)

In [79]:
df.show()

+------+------+----------+
|emp_no|salary| from_date|
+------+------+----------+
| 10001| 60117|2021-01-01|
| 10001| 62102|2022-01-01|
| 10002| 66074|2021-01-01|
| 10002| 66596|2022-01-01|
| 10003| 22222|2022-01-01|
| 10004|  3000|2022-01-01|
+------+------+----------+



In [80]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag,when, round

In [81]:
window_spec = Window\
.partitionBy('emp_no')\
.orderBy('from_date')

In [82]:
df = df.withColumn('lagby1',lag(df.salary).over(window_spec))

In [83]:
df = df.withColumn('lagby1',when(df.lagby1.isNull(),df.salary)\
                   .otherwise(df.lagby1))

In [84]:
df = df.withColumn('percentagersie',round((((df.salary-df.lagby1)/df.salary)*100),2))\
      .drop('lagby1')

In [85]:
df.show()

+------+------+----------+--------------+
|emp_no|salary| from_date|percentagersie|
+------+------+----------+--------------+
| 10001| 60117|2021-01-01|           0.0|
| 10001| 62102|2022-01-01|           3.2|
| 10002| 66074|2021-01-01|           0.0|
| 10002| 66596|2022-01-01|          0.78|
| 10003| 22222|2022-01-01|           0.0|
| 10004|  3000|2022-01-01|           0.0|
+------+------+----------+--------------+

